In [29]:
%matplotlib inline
import pickle
import pandas as pd
from IPython.display import display

# Loading 2017 train data

In [30]:
with open('../data/processed/data.pkl', 'rb') as pf:
    objs = pickle.load(pf)

grammar_dic = objs[0]
df_17_train = objs[1]   # using RecResult
df_17_test = objs[2]    # using RecResult

recode = lambda x: 1 if x=='correct' else 0
df_17_train['language'] = df_17_train['language'].apply(recode)
df_17_test['language'] = df_17_test['language'].apply(recode)
df_17_train['meaning'] = df_17_train['meaning'].apply(recode)
df_17_test['meaning'] = df_17_test['meaning'].apply(recode)

# load various types of features

# df_17_train_vecsim = pd.read_csv('../data/processed/df17_train_fasttext.csv')
df_17_train_grmerr = pd.read_csv('../data/processed/df17_train_grmerror.csv')
# df_17_test_vecsim = pd.read_csv('../data/processed/df17_test_fasttext.csv')
df_17_test_grmerr = pd.read_csv('../data/processed/df17_test_grmerror.csv')

print(df_17_train.head(5))
print(df_17_test.shape)
# print(df_17_test_vecsim.shape)
print(df_17_test_grmerr.shape)

     Id                                  Prompt   Wavfile  \
0  5835               Frag: Zimmer für 4 Nächte  5835.wav   
1  5836                      Frag: Junior Suite  5836.wav   
2  5837                      Frag: Junior Suite  5837.wav   
3  5839  Sag: Ich möchte mit Postkarte bezahlen  5839.wav   
4  5840           Frag: Gibt es einen Coiffeur?  5840.wav   

                          RecResult                     Transcription  \
0            a room for four nights            a room for four nights   
1                      junior suite                    junior*z suite   
2                            junior                    kinderzimmer*v   
3  i would like to pay by post card  i would like to pay by post card   
4            is there a hairdresser            is there a hairdresser   

   language  meaning  
0         1        1  
1         1        1  
2         0        0  
3         1        1  
4         1        1  
(995, 7)
(993, 12)


## Form a DF for modeling
- df_17_X, only using Id, language, and meaning cols
- using Id to merge other features DFs


- 1/16/2018 Add more features provided by Huy
- 1/24/2018 Just keep one DF for langauge prediction
- 1/24/2018 Added updated grammar error count features (based on kaldi ASR)

In [31]:
df_17_train_huy = pd.read_csv('../data/processed/Huy/textProcessing_trainingKaldi_features.csv', sep='\t')
df_17_test_huy = pd.read_csv('../data/processed/Huy/textProcessing_testKaldi_annotated_features.csv',
                          sep = '\t')
df_17_train_huy = df_17_train_huy.drop(['CLASS'], axis=1)
df_17_test_huy = df_17_test_huy.drop(['CLASS'], axis=1)
print(df_17_train_huy.shape)
print(df_17_train_huy.columns)
print(df_17_test_huy.shape)

(5222, 50)
Index(['Id', 'ppl-ref', 'ppl-ref_pos', 'ppl-ref_prod', 'ppl-ref_dep',
       'ppl-prompt', 'ppl-prompt_pos', 'ppl-correct', 'ppl-correct_pos',
       'ppl-correct_prod', 'ppl-correct_dep', 'ppl-ge', 'ppl-ge_pos',
       'ppl-incorrect', 'ppl-incorrect_pos', 'ppl-incorrect_prod',
       'ppl-incorrect_dep', 'maxsim_15_skip', 'maxsim_30_skip',
       'maxsim_50_skip', 'maxsim_15_cbw', 'maxsim_30_cbw', 'maxsim_50_cbw',
       'lda_sim-max', 'lda_sim-min', 'lda_sim-avg', 'ngram_match',
       'ngram_match-lem', 'ngram_unseen-1', 'ngram_unseen-2', 'ngram_unseen-3',
       'error_yn', 'error_count', 'error_ratio', 'parse_score-ratio',
       'length_ratio', 'length_01', 'length_under-min', 'length_above-max',
       'length_unknown', 'length_unknown-ratio', 'length_sounds',
       'length_sounds-ratio', 'prompt_missing', 'prompt_missing-pct',
       'prompt_DT', 'prompt_IN', 'prompt_MD', 'prompt_NN', 'prompt_VB'],
      dtype='object')
(995, 50)


In [32]:
def gen_ml_df(df_main, df_grmerr, df_huy):
    df_ml = df_main[['Id', 'language', 'meaning']]
    df_ml = pd.merge(df_ml, df_grmerr, on='Id')
    df_ml = pd.merge(df_ml, df_huy, on='Id')
    return df_ml

## Process 2017 train DF

In [33]:
df_17_train_ml = gen_ml_df(df_17_train, df_17_train_grmerr, df_17_train_huy)
display(df_17_train_ml.describe())
print(df_17_train_ml.shape)


,Id,language,meaning,ofto_error_count,Spell,Article_a_an_error_count,sva_error_count,RuleRepeatedLinkWords_error_count,Verbform_error_count,noun_error_count,...,length_unknown-ratio,length_sounds,length_sounds-ratio,prompt_missing,prompt_missing-pct,prompt_DT,prompt_IN,prompt_MD,prompt_NN,prompt_VB
count,10439.000000,10439.000000,10439.000000,10439.0,10439.0,10439.000000,10439.000000,10439.0,10439.000000,10439.000000,...,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000,10439.000000
mean,8798.107098,0.743175,0.896829,0.0,0.0,0.001341,0.014369,0.0,0.000958,0.023565,...,0.041316,5.515088,0.793563,1.472746,0.311417,0.154804,0.149344,0.088706,0.315835,0.279146
std,1694.861387,0.436903,0.304196,0.0,0.0,0.036599,0.119013,0.0,0.030937,0.152956,...,0.122428,1.870787,0.404767,1.576254,0.324739,0.361735,0.356444,0.284332,0.464870,0.448601
min,5835.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7344.500000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8832.000000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,5.000000,1.000000,1.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10262.000000,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,7.000000,1.000000,2.000000,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000
max,11676.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,0.0,1.000000,2.000000,...,1.000000,20.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(10439, 63)


## Process 2017 test DF

In [34]:
df_17_test_ml = gen_ml_df(df_17_test, df_17_test_grmerr, df_17_test_huy)
display(df_17_test_ml.describe())
print(df_17_test_ml.shape)

,Id,language,meaning,ofto_error_count,Spell,Article_a_an_error_count,sva_error_count,RuleRepeatedLinkWords_error_count,Verbform_error_count,noun_error_count,...,length_unknown-ratio,length_sounds,length_sounds-ratio,prompt_missing,prompt_missing-pct,prompt_DT,prompt_IN,prompt_MD,prompt_NN,prompt_VB
count,993.000000,993.000000,993.000000,993.000000,993.0,993.000000,993.000000,993.0,993.000000,993.000000,...,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000,993.000000
mean,4149.993958,0.720040,0.880161,0.003021,0.0,0.001007,0.021148,0.0,0.002014,0.032226,...,0.051865,5.574018,0.720040,1.627392,0.347634,0.167170,0.118832,0.089627,0.367573,0.307150
std,325.213178,0.449205,0.324937,0.054910,0.0,0.031734,0.150791,0.0,0.044856,0.176688,...,0.116119,2.084120,0.449205,1.645539,0.333340,0.373316,0.323754,0.285791,0.482387,0.461545
min,3571.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3869.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4152.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,5.000000,1.000000,1.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4436.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,7.000000,1.000000,3.000000,0.571429,0.000000,0.000000,0.000000,1.000000,1.000000
max,4701.000000,1.000000,1.000000,1.000000,0.0,1.000000,2.000000,0.0,1.000000,1.000000,...,0.833333,27.000000,1.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(993, 63)


## Correlation analysis
Only show |R| more than $CORR_{CUT}$

In [35]:
tb_corr = df_17_train_ml.corr()
CORR_CUT = 0.1

tb_l = tb_corr['language'].drop(['language', 'meaning'])
display(tb_l[tb_l > CORR_CUT])
display(tb_l[tb_l < -1 * CORR_CUT])
col_l = list(tb_l[tb_l > CORR_CUT].index) + list(tb_l[tb_l < -1 * CORR_CUT].index)
display(col_l)
len(col_l)

ppl-ref                0.412490
ppl-ref_pos            0.253558
ppl-ref_prod           0.150825
ppl-ref_dep            0.255316
ppl-prompt             0.161957
ppl-prompt_pos         0.158707
ppl-correct            0.449567
ppl-correct_pos        0.336328
ppl-correct_prod       0.203240
ppl-correct_dep        0.110361
ppl-ge                 0.102008
ppl-ge_pos             0.115483
ppl-incorrect_prod     0.110550
ppl-incorrect_dep      0.110361
maxsim_15_skip         0.294504
maxsim_30_skip         0.303735
maxsim_50_skip         0.300757
maxsim_15_cbw          0.298570
maxsim_30_cbw          0.308608
maxsim_50_cbw          0.306712
lda_sim-max            0.240804
lda_sim-min            0.111287
lda_sim-avg            0.180160
ngram_match            0.174627
ngram_match-lem        0.165228
length_sounds-ratio    0.104849
Name: language, dtype: float64

noun_error_count      -0.150272
Article_error_count   -0.233168
ppl-incorrect         -0.134898
ngram_unseen-1        -0.367605
ngram_unseen-2        -0.417048
length_under-min      -0.126214
length_above-max      -0.125457
length_sounds         -0.141598
prompt_missing        -0.369983
prompt_missing-pct    -0.280274
prompt_IN             -0.269826
prompt_MD             -0.134329
prompt_NN             -0.266155
prompt_VB             -0.223683
Name: language, dtype: float64

['ppl-ref',
 'ppl-ref_pos',
 'ppl-ref_prod',
 'ppl-ref_dep',
 'ppl-prompt',
 'ppl-prompt_pos',
 'ppl-correct',
 'ppl-correct_pos',
 'ppl-correct_prod',
 'ppl-correct_dep',
 'ppl-ge',
 'ppl-ge_pos',
 'ppl-incorrect_prod',
 'ppl-incorrect_dep',
 'maxsim_15_skip',
 'maxsim_30_skip',
 'maxsim_50_skip',
 'maxsim_15_cbw',
 'maxsim_30_cbw',
 'maxsim_50_cbw',
 'lda_sim-max',
 'lda_sim-min',
 'lda_sim-avg',
 'ngram_match',
 'ngram_match-lem',
 'length_sounds-ratio',
 'noun_error_count',
 'Article_error_count',
 'ppl-incorrect',
 'ngram_unseen-1',
 'ngram_unseen-2',
 'length_under-min',
 'length_above-max',
 'length_sounds',
 'prompt_missing',
 'prompt_missing-pct',
 'prompt_IN',
 'prompt_MD',
 'prompt_NN',
 'prompt_VB']

40

In [36]:
# convert pandas DF to numpy array

def get_langauge_X(df):
    X = df.loc[:, col_l].values
    return X

def get_langauge_y(df):
    return df['language'].values

def get_meaning_y(df):
    return df['meaning'].values

## z-norm all features

In [37]:
from sklearn.preprocessing.data import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1)) # for SVM
#scaler = StandardScaler()

In [38]:
year17_lang_train_X = get_langauge_X(df_17_train_ml)
year17_lang_train_y = get_langauge_y(df_17_train_ml)
year17_lang_test_X = get_langauge_X(df_17_test_ml)
year17_lang_test_y = get_langauge_y(df_17_test_ml)

scaler.fit(year17_lang_train_X)
year17_lang_train_X = scaler.transform(year17_lang_train_X)
year17_lang_test_X = scaler.transform(year17_lang_test_X)

year17_meaning_train_y = get_meaning_y(df_17_train_ml)
year17_meaning_test_y = get_meaning_y(df_17_test_ml)

print(year17_lang_train_X.shape)
print(year17_lang_train_y.shape)
print(year17_lang_test_X.shape)
print(year17_lang_test_y.shape)

(10439, 40)
(10439,)
(993, 40)
(993,)


Pickle all year17 numpy arraies

In [39]:
with open('../data/processed/numpy/year17_withHuy.pkl', 'wb') as pf:
    pickle.dump([year17_lang_train_X,
                 year17_lang_train_y,
                 year17_lang_test_X,
                 year17_lang_test_y,
                 year17_meaning_train_y,
                 year17_meaning_test_y], pf)
